## Installation des packages permettant la manipulation :
- `dotenv` : Chargement des variables d'environnement
- `requests` : Effectuer des requêtes HTTP
- `pandas` : Créer et manipuler des DataFrames

In [37]:
%pip install dotenv
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importation des packages permettant la manipulation :
- Chargement des variables d'environnement avec `dotenv`
- Récupération des variables d'environnement avec `os`
- Éxécution de requête HTTP avec `requests`
- Récupération et manipulation des dates avec `datetime`
- Création et manipulation des DataFrames avec `pandas`

In [38]:
import dotenv
import os
import requests
import datetime
import pandas

## Chargement et récupération des variables d'environnement

In [39]:
dotenv.load_dotenv()

NOTION_TOKEN = os.getenv("NOTION_TOKEN")
DB_INTERVENTIONS_ID = os.getenv("DB_INTERVENTIONS_ID")
DB_INVOICES_ID = os.getenv("DB_INVOICES_ID")

print( f"DB_INTERVENTIONS_ID : {DB_INTERVENTIONS_ID}\n"
      f"DB_INVOICES_ID : {DB_INVOICES_ID}")

DB_INTERVENTIONS_ID : 20f2cdbb475781db89c4c213335e99f4
DB_INVOICES_ID : 20f2cdbb475781e7977bfdf28a148db6


# Étape 1 : Définir les entêtes pour l'API Notion

In [40]:
HEADERS = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

Création d'une fonction permettant de récupérer les propriétés d'une base de données Notion à partir de son ID

In [41]:
def get_database_properties(database_id: str):
    url = f"https://api.notion.com/v1/databases/{database_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    return response.json()

In [42]:
get_database_properties(DB_INTERVENTIONS_ID)

{'object': 'database',
 'id': '20f2cdbb-4757-81db-89c4-c213335e99f4',
 'cover': None,
 'icon': {'type': 'external',
  'external': {'url': 'https://www.notion.so/icons/checkmark_green.svg'}},
 'created_time': '2025-06-11T20:24:00.000Z',
 'created_by': {'object': 'user',
  'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
 'last_edited_by': {'object': 'user',
  'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
 'last_edited_time': '2025-06-11T20:24:00.000Z',
 'title': [{'type': 'text',
   'text': {'content': 'Suivi des tâches', 'link': None},
   'annotations': {'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False,
    'code': False,
    'color': 'default'},
   'plain_text': 'Suivi des tâches',
   'href': None}],
 'description': [{'type': 'text',
   'text': {'content': 'Restez organisé·e avec vos tâches, à votre façon.',
    'link': None},
   'annotations': {'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False,
    'code': Fals

In [43]:
get_database_properties(DB_INVOICES_ID)

{'object': 'database',
 'id': '20f2cdbb-4757-81e7-977b-fdf28a148db6',
 'cover': None,
 'icon': None,
 'created_time': '2025-06-11T20:24:00.000Z',
 'created_by': {'object': 'user',
  'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
 'last_edited_by': {'object': 'user',
  'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
 'last_edited_time': '2025-06-11T20:24:00.000Z',
 'title': [{'type': 'text',
   'text': {'content': 'Invoices', 'link': None},
   'annotations': {'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False,
    'code': False,
    'color': 'default'},
   'plain_text': 'Invoices',
   'href': None}],
 'description': [],
 'is_inline': True,
 'properties': {'Mois': {'id': 'Jh%5EE',
   'name': 'Mois',
   'type': 'rich_text',
   'rich_text': {}},
  'Invoice Number': {'id': 'NoqM',
   'name': 'Invoice Number',
   'type': 'rich_text',
   'rich_text': {}},
  'Total Amount': {'id': 'xFzN',
   'name': 'Total Amount',
   'type': 'number',
   'number': {'format':

# Étape 2 : Fonction `query_unbilled_entries(date_begin: str, date_end: str, billed: bool)`

In [44]:
def query_unbilled_entries(date_begin: str, date_end: str, billed: bool = None):
    date_begin = datetime.datetime.strptime(date_begin, "%d/%m/%Y").isoformat()
    date_end = datetime.datetime.strptime(date_end, "%d/%m/%Y").isoformat()

    base_filters = [
        {"property": "Date de début", "date": {"on_or_after": f"{date_begin}"}}
    ]

    if billed is not None:
        base_filters.append({"property": "Facturé", "checkbox": {"equals": billed}})

    query = {
        "filter": {
            "and": base_filters + [{
                "or": [
                    {"property": "Date de fin", "date": {"before": f"{date_end}"}},
                    {"property": "Date de fin", "date": {"is_empty": True}}
                ]
            }]
        }
    }

    response = requests.post(
        f"https://api.notion.com/v1/databases/{DB_INTERVENTIONS_ID}/query",
        headers=HEADERS,
        json=query,
    )
    response.raise_for_status()
    return response.json()['results']


Exécution de la fonction permettant de récupérer les entrées non facturées

In [45]:
results = query_unbilled_entries("01/01/2023", "01/01/2026", False)
results

[{'object': 'page',
  'id': '20f2cdbb-4757-818b-9f45-d151fc44e363',
  'created_time': '2025-06-11T20:24:00.000Z',
  'last_edited_time': '2025-06-11T20:24:00.000Z',
  'created_by': {'object': 'user',
   'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
  'last_edited_by': {'object': 'user',
   'id': '2f589f48-fe35-4317-8f84-345e601d4a44'},
  'cover': None,
  'icon': {'type': 'external',
   'external': {'url': 'https://www.notion.so/icons/checkmark_green.svg'}},
  'parent': {'type': 'database_id',
   'database_id': '20f2cdbb-4757-81db-89c4-c213335e99f4'},
  'archived': False,
  'in_trash': False,
  'properties': {'Ville': {'id': '%3Aso%7C',
    'type': 'select',
    'select': {'id': 'f19b6eb9-8259-4e8b-bf70-29e154dfac65',
     'name': 'Le Kremlin-Bicêtre',
     'color': 'default'}},
   'Niveau d’effort': {'id': '%3BoM%7C',
    'type': 'select',
    'select': {'id': '1993545d-653e-48d0-9496-cfc698410019',
     'name': 'Élevé',
     'color': 'red'}},
   'Classe': {'id': '%3D%3C%60%40',
    't

Récupération depuis la réponse JSON, des valeurs utiles

In [46]:
results_json = []
for result in results:
    results_json.append({
        "Ville": result['properties']['Ville']['select']['name'],
        "Classe": result['properties']['Classe']['select']['name'],
        "Date de fin": result['properties']['Date de fin']['date']['start'] if result['properties']['Date de fin']['date'] else None,
        "Total": result['properties']['Total']['formula']['number'],
        "Ecole": result['properties']['Ecole']['select']['name'],
        "Nombre heures": result['properties']['Nombre heures']['number'],
        "Facturé": result['properties']['Facturé']['checkbox'],
        "Tarif horaire": result['properties']['Tarif horaire']['number'],
        "A facturer": True if result['properties']['A facturer']['select']['name'] == "True" else False,
        "Date de début": result['properties']['Date de début']['date']['start'],
        "Cours": result['properties']['Cours']['title'][0]['plain_text'],
    })
results_json

[{'Ville': 'Le Kremlin-Bicêtre',
  'Classe': 'M1',
  'Date de fin': None,
  'Total': 70,
  'Ecole': 'EPITECH Digital',
  'Nombre heures': 7,
  'Facturé': False,
  'Tarif horaire': 10,
  'A facturer': True,
  'Date de début': '2025-03-19',
  'Cours': ' Innovation en IA & Transformation des organisations partie 2'},
 {'Ville': 'Evry',
  'Classe': 'M1 Maths',
  'Date de fin': None,
  'Total': 35,
  'Ecole': 'ENSIIE',
  'Nombre heures': 3.5,
  'Facturé': False,
  'Tarif horaire': 10,
  'A facturer': False,
  'Date de début': '2025-07-02',
  'Cours': 'Régression linéaire session 3'},
 {'Ville': 'Paris',
  'Classe': 'M1',
  'Date de fin': None,
  'Total': 70,
  'Ecole': 'ECE',
  'Nombre heures': 7,
  'Facturé': False,
  'Tarif horaire': 10,
  'A facturer': True,
  'Date de début': '2025-03-05',
  'Cours': 'Innovation en IA & Transformation des organisations partie 1'},
 {'Ville': 'Le Kremlin-Bicêtre',
  'Classe': 'M1',
  'Date de fin': None,
  'Total': 70,
  'Ecole': 'EPITECH Digital',
  'No

Création d'un DataFrame pour afficher les résultats

In [47]:
df = pandas.DataFrame(results_json)
df

,Ville,Classe,Date de fin,Total,Ecole,Nombre heures,Facturé,Tarif horaire,A facturer,Date de début,Cours
0,Le Kremlin-Bicêtre,M1,None,70.0,EPITECH Digital,7.0,False,10,True,2025-03-19,Innovation en IA & Transformation des organis...
1,Evry,M1 Maths,None,35.0,ENSIIE,3.5,False,10,False,2025-07-02,Régression linéaire session 3
2,Paris,M1,None,70.0,ECE,7.0,False,10,True,2025-03-05,Innovation en IA & Transformation des organisa...
3,Le Kremlin-Bicêtre,M1,None,70.0,EPITECH Digital,7.0,False,10,True,2025-03-05,Innovation en IA & Transformation des organisa...
4,Paris,L3,None,52.5,ECE,3.5,False,15,True,2025-05-16,Mathématiques pour l’informatique session 2
5,Paris,BTS SIO,None,52.5,NEXA Digital School,3.5,False,15,True,2025-05-23,Mathématiques pour l’informatique session 2
6,Paris,BTS SIO,None,52.5,NEXA Digital School,3.5,False,15,False,2025-07-09,Mathématiques pour l’informatique session 3
7,Noisy-le-Grand,M2 EDWEB,2025-05-03,140.0,ESIEE,28.0,False,5,True,2025-04-25,Programmation avec Python - Introduction
8,Evry,M1 Maths,None,17.5,ENSIIE,3.5,False,5,False,2025-08-07,Mathématiques pour l’informatique session 4
9,Paris,M1,None,70.0,ECE,7.0,False,10,True,2025-03-19,Innovation en IA & Transformation des organis...


## Par ville, le nombre d’écoles, le nombre d’heures données et la somme à facturer

In [48]:
df.groupby(['Ville']).agg({
    'Ecole': 'nunique',
    'Nombre heures': 'sum',
    'A facturer': 'sum'
}).reset_index()

,Ville,Ecole,Nombre heures,A facturer
0,Evry,1,56.5,4
1,Le Kremlin-Bicêtre,1,21.0,3
2,Noisy-le-Grand,1,91.0,1
3,Paris,2,67.0,6


## Par école et par classe, le nombre d’heures enseignées

In [49]:
df.groupby(['Ecole', 'Classe']).agg({
    'Nombre heures': 'sum'
}).reset_index()

,Ecole,Classe,Nombre heures
0,ECE,BTS SIO,3.5
1,ECE,L3,30.0
2,ECE,M1,14.0
3,ENSIIE,L3,23.0
4,ENSIIE,M1 EDWEB,14.0
5,ENSIIE,M1 Maths,19.5
6,EPITECH Digital,M1,21.0
7,ESIEE,M1,63.0
8,ESIEE,M2 EDWEB,28.0
9,NEXA Digital School,BTS SIO,19.5
